In [2]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [6]:
# today = date.today().strftime('%Y%m%d')
today = 20230610

input_folder = "author_flair"
input_file = "author_flair"
mode = "append"

df_raw = spark.read.option("header", "true") \
    .json(f"s3a://{minio_bucket}/bronze/popular_{today}.json")

In [7]:
df_raw = df_raw.select(explode(df_raw.data.children.data).alias("data"))
df_raw = df_raw.select("data.*")

In [8]:
df_raw.show()

+--------------------+-------------------+---------------+-----------+--------+--------------------+--------------+-----------------------------+----------------------+---------------------+------------------------+-----------------+-----------------------+-----------------+---------------+-----------------+--------------------+--------------+--------+-------------+---------+--------+------------+--------+-------+------------------+------------+-------------+-------------+---------------+-------------+--------------------+-----+-------------+------------+------+------------------+------+----------+-------+----------------------+----------------+----------+-------+-------------------+----------------------+------------------+-------+--------+-----+---------------------------+--------------------+--------------------+----------------------+--------------------+---------------------+---------------+------+--------------------+--------------------+--------------+----------+--------+-------

In [9]:
df_raw.printSchema()

root
 |-- all_awardings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- award_sub_type: string (nullable = true)
 |    |    |-- award_type: string (nullable = true)
 |    |    |-- awardings_required_to_grant_benefits: string (nullable = true)
 |    |    |-- coin_price: long (nullable = true)
 |    |    |-- coin_reward: long (nullable = true)
 |    |    |-- count: long (nullable = true)
 |    |    |-- days_of_drip_extension: long (nullable = true)
 |    |    |-- days_of_premium: long (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- end_date: string (nullable = true)
 |    |    |-- giver_coin_reward: string (nullable = true)
 |    |    |-- icon_format: string (nullable = true)
 |    |    |-- icon_height: long (nullable = true)
 |    |    |-- icon_url: string (nullable = true)
 |    |    |-- icon_width: long (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- is_enabled: boolean (nullable